In [1]:
"""

LIWC is a dictionary which consist of categories and associated words to it.

I match song by song with the LIWC data and store data which overlaps b/w my song and one of their categories

1. def get_liwc_sum returns the sum of words in one liwc category on song level
2. def get_liwc_words returns the words per cat on a song level

sidenote: I added/deleted words to the "she" and "he" category as it seemed they were very outdated

"""






import pandas as pd
import csv
from collections import defaultdict
import string
import pandas as pd
import sys
import datetime
import os
# from pprint import pprint 
# pp = pprint.PrettyPrinter(indent=4)


class LIWC_Index_Count(): # Very old style class defined :)
    def category_matches(self, doc, categories):
        all_words = {}
        time = datetime.datetime.now()
        # print "\nContinue with checking categories, can take some time: %s\n" % (time)
        ''' Returns a mapping of LIWC category -> number of occurrences in the given doc '''
        num_matches = defaultdict(int) # Unused variable 'num_matches'
        
        doc = str(doc)
        words = doc.lower().split(' ')
        words_no_punct = [''.join(ch for ch in token if not ch in string.punctuation) for token in words] 
        for w in words_no_punct:
            for category in categories:
                category_stems = self.LIWC_cat_to_stems[category]
                for stem in category_stems:
                    matches_cat = w.startswith(stem.replace("*","")) if stem.endswith("*") else (stem==w)
                    if matches_cat:
                      word = w
                      bucket = category
                      if category in all_words.keys():
                        all_words[bucket][word] = all_words[bucket].get(word, 0) + 1
                      else:
                        all_words[bucket] =  {w : 1}
                      break # done with this category so move to the next        
       
        # print "Done with category check. Finished in: %s\n" % (datetime.datetime.now()- time)
        return all_words
    
    def __init__(self):
        (self.LIWC_stem_to_cats, self.LIWC_cat_to_stems) = self.__init_LIWC_dict__()
    def __init_LIWC_dict__(self):   # Method could be a function :-|
        cat_col_map = {
            'Funct':[0,1,2],
            'Pronoun':[3],
           'Ppron':[4],
           'I':[5],
           'We':[6],
           'You':[7],
           'She':[8],
           'They':[9],
           'He':[10],
           'CoreShe':[11],
           'CoreHe':[12],
           'See':[13],
           'Hear':[14],
           'Feel':[15],
           'Body':[16, 17],
           'Health':[18, 19],
           'Sexual':[20],
           'Work':[21, 22],
           'Achiev':[23, 24],
           'Leisure':[25, 26],
           'Home':[27],
           'Money':[28, 29],
           'Social':[30, 31],
           'Family': [32],
           'Friends': [33],
           'Colour': [34]
           }
        LIWC_stem_to_cats = defaultdict(list)
        LIWC_cat_to_stems = defaultdict(list)
#         LIWC_dictionary = os.path.join(os.path.dirname(__file__), "./resources/LIWC2007dictionary.csv")
        LIWC_dictionary = "./resources/thisisgenderLIWC.csv"
        
        row_count = -1
        for row in csv.reader(open(LIWC_dictionary, "rU")):
            row_count+=1
            if row_count<3 or row_count>171:
                continue # header rows or past any content
            for cat in cat_col_map:
                cols = cat_col_map[cat]
                for col in cols:
                    wp = row[col]
                    if wp == "":
                        continue
                    LIWC_stem_to_cats[wp].append(cat)
                    LIWC_cat_to_stems[cat].append(wp)
        return (LIWC_stem_to_cats, LIWC_cat_to_stems)


In [2]:
class RevealWordsOfPosts(object):

    def __init__(self):
        pass

    def getLIWCsum(self, String, Category):
        num_processed = 0
        time = datetime.datetime.now()
        file_content = String
        new = LIWC_Index_Count()
        count_whole_corpus = new.category_matches(file_content, [Category])
        df = pd.DataFrame.from_dict(count_whole_corpus, orient='columns', dtype=None)
        try:
          dict_cat =  df.to_dict()[Category] 
          sum_num = sum(dict_cat.values())
          return sum_num
        except KeyError:
            return 0


    def getLIWCwords(self, String, Category):
        num_processed = 0
        time = datetime.datetime.now()
        file_content = String
        new = LIWC_Index_Count()
        count_whole_corpus = new.category_matches(file_content, [Category])
        df = pd.DataFrame.from_dict(count_whole_corpus, orient='columns', dtype=None)
        try:
            values =  df.to_dict()[Category]
            return values.items()
        except KeyError:
            return 0

In [4]:
'''
Retrieving the Sum of Categories
'''

df = pd.read_csv("./raw-data/billboard_lyrics_1964-2015.csv")
print("START")
df_sum = pd.DataFrame()
df_sum["Lyrics"] = df.apply(lambda row: row["Lyrics"], axis=1)
df_sum["Year"] = df.apply(lambda row: row["Year"], axis=1)
df_sum["She"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "She"), axis=1)
df_sum["He"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "He"), axis=1)
df_sum["CoreHe"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "CoreHe"), axis=1)
df_sum["CoreShe"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "CoreShe"), axis=1)
df_sum["Achiev"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Achiev"), axis=1)
print("half")
df_sum["Work"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Work"), axis=1)
df_sum["Body"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Body"), axis=1)
df_sum["See"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "See"), axis=1)
df_sum["Hear"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Hear"), axis=1)
df_sum["Feel"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Feel"), axis=1)
df_sum["Health"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Health"), axis=1)
df_sum["Leisure"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Leisure"), axis=1)
df_sum["Home"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Home"), axis=1)
print("third")
df_sum["Money"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Money"), axis=1)
df_sum["Social"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Social"), axis=1)
df_sum["Family"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Family"), axis=1)
df_sum["Friends"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Friends"), axis=1)
df_sum["Colour"] = df.apply(lambda row: RevealWordsOfPosts().getLIWCsum(row["Lyrics"], "Colour"), axis=1)

print("END")
df_sum.to_csv("./raw-data/liwc-catSum-per-song.csv")

START


/Users/franzi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: DeprecationWarning: 'U' mode is deprecated


half
third
END


In [ ]:
"""
Retrieving the words of for each category 
"""

df_2 = pd.read_csv("./raw-data/billboard_lyrics_1964-2015.csv")
df_word["Lyrics"] = df_2.apply(lambda row: row["Lyrics"], axis=1)
df_word["Lyrics"] = df_2.apply(lambda row: row["Year"], axis=1)
df_word["She"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "She"), axis=1)
df_word["He"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "He"), axis=1)
df_word["CoreHe"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "CoreHe"), axis=1)
print("quater")
df_word["CoreShe"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "CoreShe"), axis=1)
df_word["Achiev"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Achiev"), axis=1)
df_word["Work"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Work"), axis=1)
df_word["Body"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Body"), axis=1)
df_word["See"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "See"), axis=1)
df_word["Hear"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Hear"), axis=1)
df_word["Feel"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Feel"), axis=1)
print("third")
df_word["Health"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Health"), axis=1)
df_word["Leisure"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Leisure"), axis=1)
df_word["Home"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Home"), axis=1)
df_word["Money"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Money"), axis=1)
df_word["Social"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Social"), axis=1)
df_word["Family"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Family"), axis=1)
print('almost')
df_word["Friends"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Friends"), axis=1)
df_word["Colour"] = df_2.apply(lambda row: RevealWordsOfPosts().getLIWCwords(row["Lyrics"], "Colour"), axis=1)

print("END")
df_sum.to_csv("./raw-data/liwc-words-per-song.csv")